In [5]:
import time

count = 0
tmp_list = []
with open("nounlist_translated.txt",'r') as f:
    for word in f:
        word = word.strip()
        if ' ' in word:
            continue
        tmp_list.append(word.lower())
        
with open("nounlist.txt",'w') as f:
    for word in sorted(list(set(tmp_list))):
        f.write(word.lower() + '\n')

In [7]:
import pyspark
import re
from nltk import word_tokenize

sc = pyspark.SparkContext('local[*]')

noun_set = set()
with open("nounlist.txt",'r') as f:
    for line in f:
        noun_set.add(line.strip().lower())
noun_set_bc = sc.broadcast(noun_set)

lines = sc.textFile("ptwiki-20190201-pages-articles.xml.bz2")
# lines = sc.parallelize(lines.take(20000)) # in case you want to try a small sample

content_rdd = lines.filter(lambda line: line and line[0].isalpha())
counts = content_rdd.flatMap(lambda line: [word.lower() for word in word_tokenize(line)]) \
            .filter(lambda word: word in noun_set_bc.value) \
            .map(lambda word: (word, 1)) \
            .reduceByKey(lambda a, b: a + b)
count_tuples = counts.sortBy(lambda a: -a[1]).collect()

words_found = []
with open("nounlist_frequency.csv",'w') as f:
    for word,count in count_tuples:
        f.write(word + ',' + str(count) + '\n')
        words_found.append(word)
    
    # make sure all the words are in the file even if never found on wikipedia
    words_found = set(words_found)
    for word in noun_set:
        if word not in words_found:
            f.write(word + ',0\n')
sc.stop()